In [3]:
%cd /home/druhe/rail1/src/cnf/
import os
os.environ['DATAROOT'] = '/home/druhe/datasets/'

/home/druhe/rail1/src/cnf


/home/druhe/rail1/src/cnf/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [24]:
from algebra.cliffordalgebra import CliffordAlgebra
import torch

In [25]:
ca = CliffordAlgebra((1, 1, 1))

In [26]:
V = ca.random_vector(3)

V_rev = torch.flip(V, [0])

In [28]:
print(V, V_rev)

tensor([[ 0.0000, -0.6764, -0.1591, -1.8175,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000, -1.6665,  0.9410,  0.4360,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.6251, -0.2713,  1.0560,  0.0000,  0.0000,  0.0000,  0.0000]]) tensor([[ 0.0000,  0.6251, -0.2713,  1.0560,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000, -1.6665,  0.9410,  0.4360,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000, -0.6764, -0.1591, -1.8175,  0.0000,  0.0000,  0.0000,  0.0000]])


In [42]:
basis = [V[0]]
for i in range(1, 3):
    wedge = ca.get_grade(ca.reduce_geometric_product(V[:i]), i)
    wedge = ca.embed_grade(wedge, i)

    wedge_wedge = ca.geometric_product(wedge, V[i])
    
    wedge_inv = ca.inverse(wedge)

    basis.append(ca.geometric_product(wedge_wedge, wedge_inv))

In [43]:
basis = torch.stack(basis)
basis = ca.get_grade(basis, 1)

In [47]:
# Is basis orthogonal?
basis @ basis.T

tensor([[ 3.7862,  0.1851,  2.2989],
        [ 0.1851,  3.8527, -0.6117],
        [ 2.2989, -0.6117,  1.5794]])

In [48]:
v = torch.randn(3, 3)

In [51]:
u = v.clone()

In [52]:
u[1] = v[1] - torch.dot(u[0], v[1]) / torch.dot(u[0], u[0]) * u[0]

In [53]:
u[2] = v[2] 

In [98]:
def gram_schmidt(v):
    u = v.clone()
    for i in range(1, len(u)):
        z = torch.einsum('ij, j->i', u[:i], v[i]) / torch.einsum('ij, ij->i', u[:i], u[:i]) 
        u[i] = v[i] - torch.einsum('i, ij->j', z, u[:i])
    u /= u.norm(dim=1, keepdim=True)
    return u

In [113]:
def gram_schmidt_b(v):
    u = v.clone()
    for i in range(1, len(u)):
        z = torch.einsum('bij, bj->bi', u[:, :i], v[:, i]) / torch.einsum('bij, bij->bi', u[:, :i], u[:, :i]) 
        u[:, i] = v[:, i] - torch.einsum('bi, bij->bj', z, u[:, :i])
    u /= u.norm(dim=2, keepdim=True)
    return u

In [114]:
e = gram_schmidt(v[None])

In [115]:
e[0] @ e[0].T

tensor([[ 1.0000, -0.8050,  0.6590],
        [-0.8050,  1.0000, -0.5031],
        [ 0.6590, -0.5031,  1.0000]])